In [21]:
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
import sys
import numpy
from keras.models import load_model
import pickle
import os
import keras
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
default_model_path = '/home/dev2/Sanjana/stride/'

model = load_model(os.path.join(default_model_path,'binary_model_new.h5'))  # loading default model
tokenizer = pickle.load(open(os.path.join(default_model_path, 'word_tokenizer_binary_model.pickle'), 'rb'))
# encoder_level = pickle.load(open(os.path.join(default_model_path, 'levels_roclass.pickle'), 'rb'))
character_tokenizer = pickle.load(open(os.path.join(default_model_path, 'character_tokenizer_binary.pickle'), 'rb'))

char2idx = character_tokenizer
word_index = tokenizer.word_index
word_index['pad'] = 0
sent_max_len = 300
word_max_len = 10
N = 1
numpy.set_printoptions(threshold=sys.maxsize)


[nltk_data] Downloading package stopwords to /home/dev2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
def intent_prediction(lista):
    empty_out=[]
    for i in lista:
        text=i
        text_tokens = word_tokenize(text)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        text= (" ").join(tokens_without_sw)
        if 'list' not in str(type(text)):
            text = [text]
            print(text)
        print("Length in for batch predict class:", len(text))
        
        text[:] = [s.lower().replace(r'[^a-zA-Z\s]+|X{2,}', '') for s in text]
        text[:] = [re.sub(r'[^\w\s]', '', s) for s in text]
        sequences = tokenizer.texts_to_sequences(text)
        padded_sequences = keras.preprocessing.sequence.pad_sequences(maxlen=sent_max_len, sequences=sequences,
                                                                      value=word_index["pad"], padding='post',
                                                                      truncating='post')
        print("check")
        character_list = []
        for para in text:
            para_list = []
            for i in range(sent_max_len):
                word_seq = []
                for j in range(word_max_len):
                    try:
                        if char2idx.get(para[i][j]) is None:
                            raise Exception
                        word_seq.append(char2idx.get(para[i][j]))
                    except:
                        word_seq.append(char2idx.get("pad"))
                para_list.append(word_seq)
            character_list.append(para_list)
        print("1")
        character_list = np.array(character_list)
        predictions = model.predict([padded_sequences,
                                            character_list.reshape(character_list.shape[0], sent_max_len,
                                                                   word_max_len)])
        print(predictions)
        classes=["No","Yes"]
        empty_out.append((classes[int(predictions.argmax(axis=1))]))
    print(len(lista),len(empty_out))
    return empty_out


In [23]:
print("done")

done


In [24]:
import pandas as pd
df =pd.read_csv(r'test.csv')
df

,Tags,paragraphs,Unnamed: 2
0,No,I look forward to meeting you and learning abo...,NaN
1,No,We look forward to seeing you next week!,NaN
2,No,A quick question before our meeting.,NaN
3,No,After sunning and drinking all day we feasted ...,NaN
4,No,"Also Tuesday, Kyle and Eric 1/2 hour.",NaN
5,No,"Also, I made a change to paragraph 7 to includ...",NaN
6,No,As i was pondering the meeting last night goin...,NaN
7,No,"As you become the responsible lawyer, first in...",NaN
8,No,"As you know, last year's congress attracted ov...",NaN
9,No,"As you may know, I am on the Compaq Computer b...",NaN


In [25]:
lista=df['paragraphs'].tolist()
output_list=intent_prediction(lista)
df['Predicted_Output']=output_list

['I look forward meeting learning successful business .']
Length in for batch predict class: 1
check
1
[[0.98066807 0.01933199]]
['We look forward seeing next week !']
Length in for batch predict class: 1
check
1
[[9.9983752e-01 1.6241612e-04]]
['A quick question meeting .']
Length in for batch predict class: 1
check
1
[[0.41632786 0.58367217]]
['After sunning drinking day feasted fleisch dinner ... eat , grilled bauchfleisch , steaks bratw ? rst ... WE ATE SOOO MUCH , DRANK SO MUCH THAT WE NEEDED IT , good greasy stuff .']
Length in for batch predict class: 1
check
1
[[9.9945897e-01 5.4099225e-04]]
['Also Tuesday , Kyle Eric 1/2 hour .']
Length in for batch predict class: 1
check
1
[[9.9981862e-01 1.8136378e-04]]
["Also , I made change paragraph 7 include technical bulletins Mitch 's request discuss call ."]
Length in for batch predict class: 1
check
1
[[0.9514244  0.04857565]]
['As pondering meeting last night going home car , I realized I forgot talk 2 people , including regulatory 

In [33]:
true_count=0
false_count=0
for row in df.itertuples():
    if row.Tags==row.Predicted_Output:
        true_count=true_count+1
    else:
        false_count=false_count+1
        
print("true_count",true_count)
print("false_count",false_count)
print("total_count",true_count+false_count)
accuracy=100*(true_count/(true_count+false_count))
print("Accuracy of model is ",accuracy)

true_count 778
false_count 214
total_count 992
Accuracy of model is  78.42741935483872


In [27]:
df.to_excel(r'/home/dev2/Sanjana/stride/output.xlsx')